In [1]:
# imports

from surfacecode.logical_qubit import LQubit
from surfacecode.circuits import SquareLattice
from surfacecode.lattice import *
from surfacecode.simulators import *
from surfacecode.surface import *
from surfacecode.mwpm import ParityCheckMatrix, ErrorCorrection

from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error, amplitude_damping_error, phase_damping_error)
from qiskit_ibm_provider import IBMProvider, least_busy

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm


In [2]:
WIDTH, HEIGHT = 3, 9
l = (7, 19)

lattice = SquareLattice(WIDTH, HEIGHT)
qubit = LQubit(lattice, l[0], l[1], True)
cycle = SurfaceCodeCycle(lattice)
num_qubits = WIDTH * HEIGHT

In [3]:
qc = ConstrainedQuantumCircuit(lattice, WIDTH * HEIGHT)
qc.add_register(ClassicalRegister(num_qubits))
qc = qc.compose(cycle._circuit(1), list(range(num_qubits)), list(range(num_qubits)))

qc.add_register(ClassicalRegister(num_qubits + len(qubit.route(l[0],l[1])) // 2))
qc = qc.compose(qubit.alt_initialize(cycle), list(range(num_qubits)), list(range(num_qubits, 2 * num_qubits + len(qubit.route(l[0],l[1])) // 2)))

qc.add_register(ClassicalRegister(num_qubits))
qc = qc.compose(cycle._circuit(1), list(range(num_qubits)), list(range(2 * num_qubits + len(qubit.route(l[0],l[1])) // 2, 3 * num_qubits + len(qubit.route(l[0],l[1])) // 2)))

qc = qc.compose(qubit.circle_gate())

qc.add_register(ClassicalRegister(num_qubits))
qc = qc.compose(cycle._circuit(1), list(range(num_qubits)), list(range(3 * num_qubits + len(qubit.route(l[0],l[1])) // 2, 4 * num_qubits + len(qubit.route(l[0],l[1])) // 2)))

#qc.add_register(ClassicalRegister(num_qubits))
#qc = qc.compose(cycle._circuit(1), list(range(num_qubits)), list(range(4 * num_qubits + len(qubit.route(l[0],l[1])) // 2, 5 * num_qubits + len(qubit.route(l[0],l[1])) // 2)))

qc.add_register(ClassicalRegister(2))
qc = qc.compose(qubit.alt_measure(), list(range(num_qubits)), list(range(4 * num_qubits + len(qubit.route(l[0],l[1])) // 2, 4 * num_qubits + len(qubit.route(l[0],l[1])) // 2 + 2)))

In [6]:
noise_model = NoiseModel()
error_amplitude = amplitude_damping_error(0.5)
error_phase = phase_damping_error(0.1)
noise_model.add_all_qubit_quantum_error(error_amplitude, ['x', 'z', 'h'])

aersim = AerSimulator(method="statevector", noise_model=noise_model)
built = transpile(qc, aersim)
job = aersim.run(built, shots=1)
result = job.result()

In [7]:
result.get_counts()

{'11 000000000000010000000000000 000100000000010000000101000 00010000000100010000000000010 000000000000000000000000000': 1}

In [ ]:
10 - initilzation and measure
10 - single qubit gate